# ImmoEliza - Data Modelling

This notebook explores the different data models and how data selection and scaling impacts the predicted price. 

If you want to rerun the model for a different property type (see below), you will need to import the dataframe again.

**Caveats:**
* At the moment this model only includes numerical variables (such as number of bedrooms or area of the plot) not categorical variables (such as building condition, heating type or EPC). 
* The model does not use any form of scaling.

In [1]:
#import libraries 
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px

In [2]:
#import dataframe and display first rows and the total number of columns and rows
df = pd.read_csv('../data/saved_data_myt.csv', index_col=0)
display(df.head())
df.shape

,property_type,property_subtype,immo code,Location,Postcode,Province,Region,Price,Address,Energy class,...,Bedrooms,Bathrooms,Shower rooms,Office,Toilets,Kitchen type,Heating type,Price2,Price_orig,Terrace2
0,house,house,10666428,essen,2910,Antwerpen,Flanders,469000,Heidestatiestraat 26,D,...,3,1,0,0,2,Installed,0,NaN,€469000,NaN
1,house,house,10666426,bierges,1301,Brabant wallon,Wallonia,1395000,"Rue de Wavre, 27",C,...,5,2,3,1,4,Hyper equipped,Fuel oil,NaN,€1395000,NaN
2,apartment,apartment,10666424,arlon,6700,Luxembourg,Wallonia,285000,Avenue de Longwy 340,D,...,2,2,0,0,2,Installed,Electric,NaN,€285000,NaN
3,house,house,10666423,arlon,6700,Luxembourg,Wallonia,285000,Avenue de Longwy 340,D,...,2,2,0,0,2,Installed,Electric,NaN,€285000,NaN
4,apartment,apartment,10666422,antwerp,2000,Antwerpen,Flanders,425000,"Grote steenweg, 511",C,...,2,1,0,0,0,Installed,0,NaN,€425000,NaN


(6623, 29)

### Select properties by (sub)type
You can hardly compare a kot to a castle. Therefore you can run the model for different property (sub)types. This model is limited to housing types so commercial and industrial buildings are excluded. It uses the types and subtypes set by Immoweb. There are two main types, 'house' and 'apartment' with the following subtypes: 

* house: house, villa, mixed-use-building, exceptional-property, other-property, country-cottage, mansion, town-house, bungalow, farmhouse, new-real-estate-project-houses, chalet, manor-house, castle
* apartment: apartment, service-flat, duplex, triplex, apartment-block, loft, ground-floor, new-real-estate-project-apartments, kot, penthouse, flat-studio

By changing the codeblock below, you can select which (sub)types you want to include (==) or exclude (!=). By default castles are excluded from the model. If you want to change this selection, you should start by reloading the original data by rerunning the codeblock above.

In [3]:

#df = df[df.property_type == 'apartment']
df = df[df.property_type == 'house']
df = df[df.property_subtype != 'castle'] #by default we exclude castles from our model 

In [4]:
print(df.shape)
display(df.head())

(4196, 29)


,property_type,property_subtype,immo code,Location,Postcode,Province,Region,Price,Address,Energy class,...,Bedrooms,Bathrooms,Shower rooms,Office,Toilets,Kitchen type,Heating type,Price2,Price_orig,Terrace2
0,house,house,10666428,essen,2910,Antwerpen,Flanders,469000,Heidestatiestraat 26,D,...,3,1,0,0,2,Installed,0,NaN,€469000,NaN
1,house,house,10666426,bierges,1301,Brabant wallon,Wallonia,1395000,"Rue de Wavre, 27",C,...,5,2,3,1,4,Hyper equipped,Fuel oil,NaN,€1395000,NaN
3,house,house,10666423,arlon,6700,Luxembourg,Wallonia,285000,Avenue de Longwy 340,D,...,2,2,0,0,2,Installed,Electric,NaN,€285000,NaN
6,house,house,10666419,grace-hollogne,4460,Liï¿½ge,Wallonia,189000,Avenue Louis Libert 1,E,...,2,1,0,0,0,Semi equipped,Fuel oil,NaN,€189000,NaN
7,house,villa,10666418,aywaille,4920,Liï¿½ge,Wallonia,399999,Avenue Louis Libert 1,C,...,4,1,0,1,0,Semi equipped,Fuel oil,NaN,€399999,NaN


### Define model variables


In [ ]:
#int_columns=['Bedrooms', 'Furnished', 'Terrace', 'Terrace surface', 'Surface of the plot', 'Living room surface', 'Number of frontages', 'Construction year', 'Bathrooms', 'Shower rooms', 'Office', 'Toilets']
int_columns=['Bedrooms', 'Terrace', 'Terrace surface', 'Surface of the plot', 'Living room surface', 'Number of frontages', 'Construction year', 'Bathrooms', 'Shower rooms', 'Office', 'Toilets', 'Primary energy consumption']
#int_columns=["Bedrooms", "Bathrooms", "Toilets"]

X = df[int_columns].to_numpy()
y = df.Price.to_numpy().reshape(-1,1)
print(type(X))
print(X)


### Some graphs

In [ ]:
# create a dataframe which (only) combines X & y to create some graphs
cropped_df = df[int_columns] + df["Price"]

In [ ]:
"""Create pairplots (versus Price) and display dataframe"""

"""display(cropped_df.sample(10))
sns.pairplot(data=df, y_vars = "Price", x_vars = int_columns, height=5)
sns.pairplot(data=cropped_df)"""

In [ ]:
"""Create heatmaps for full dataframe and selected columns"""

sns.heatmap(df.select_dtypes(include="number").corr(), cmap="YlGnBu", annot=False) #heatmap of all numerical columns in dataframe
plt.show

In [ ]:
sns.heatmap(cropped_df.corr(), cmap="YlGnBu", annot=True) #heatmap of selected columns
plt.show

### Split the dataset

You now know the process!

**Exercise :** Import `train_test_split` from `sklearn` and split the dataset and create the variables `X_train`, `X_test`, `y_train`, `y_test`.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=41)

In [ ]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

##  Load and fit the model (with Sklearn)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import LinearRegression

LR_model = "Linear Regression"

LR_regressor = LinearRegression()
"""X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)"""
LR_regressor.fit(X_train, y_train)
LR_train_score = LR_regressor.score(X_train, y_train)
LR_test_score = LR_regressor.score(X_test,y_test)
LR_y_pred = LR_regressor.predict(X_test)
LR_R_MSE = mean_squared_error(y_test, LR_y_pred, squared = False)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DT_model = "Decision Tree"

DT_regressor = DecisionTreeRegressor(max_depth=6, min_weight_fraction_leaf=0.01)

DT_regressor.fit(X_train, y_train)
DT_train_score = DT_regressor.score(X_train, y_train)
DT_test_score = DT_regressor.score(X_test,y_test)
DT_y_pred = DT_regressor.predict(X_test)
DT_R_MSE = mean_squared_error(y_test, DT_y_pred, squared = False)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

GS_model = "Decision Tree with GridSearch"

params = {"max_depth" : [2, 4, 6, 8, 10, 12], "min_weight_fraction_leaf" : [0.002, 0.003, 0.0035, 0.004, 0.0045, 0.005, 0.006, 0.01]}
grid_search = GridSearchCV(DT_regressor, params, cv = 10)
grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
print(best_params)
GS_regressor = DecisionTreeRegressor(max_depth=best_params["max_depth"], min_weight_fraction_leaf=best_params["min_weight_fraction_leaf"])
GS_regressor.fit(X_train, y_train)
GS_train_score = DT_regressor.score(X_train, y_train)
GS_test_score = DT_regressor.score(X_test,y_test)
GS_y_pred = DT_regressor.predict(X_test)
GS_R_MSE = mean_squared_error(y_test, DT_y_pred, squared = False)

In [ ]:
import xgboost as xg

XG_model = "XG Boost"

XG_regressor = xg.XGBRegressor(objective ='reg:linear', n_estimators = 10, seed = 20)
"""X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)"""
XG_regressor.fit(X_train, y_train)
XG_regressor.fit(X_train, y_train)
XG_train_score = XG_regressor.score(X_train, y_train)
XG_test_score = XG_regressor.score(X_test,y_test)
XG_y_pred = XG_regressor.predict(X_test)
XG_R_MSE = mean_squared_error(y_test, XG_y_pred, squared = False)

### Scores

"Create a dataframe with an overview of the test scores of all the models"

model_list = ["LR", "DT", "GS", "XG"]
model_score_list = ["model", "train_score", "test_score", "R_MSE"]
data = []
for model in model_list:
    #print(model)
    modelscores = []
    for score in model_score_list:
        item = model + "_" + score
        modelscores.append(item)
    #print(modelscores)
    #modelscores = list(re.replace(r"/['"]+/g", '', str(modelscores)))
    #print(print(modelscores))
    data.append(modelscores)

print(data)
score_df = pd.DataFrame(data, columns=model_score_list)
score_df


In [ ]:
model_score_list = ["model", "train_score", "test_score", "R_MSE"]
#display(score_df)
#score_df["train_score_old"] = score_df["train_score"]
#score_df["test_score_old"] = score_df["test_score"]
#score_df["R_MSE_old"] = score_df["R_MSE"]

data = [[LR_model, LR_train_score, LR_test_score, LR_R_MSE], [DT_model, DT_train_score, DT_test_score, DT_R_MSE], [GS_model, GS_train_score, GS_test_score, GS_R_MSE], [XG_model, XG_train_score, XG_test_score, XG_R_MSE]]
newscore_df = pd.DataFrame(data, columns=model_score_list)
#score_df = score_df.join(newscore_df, on=['model'], how='right')
#score_df ["train_score", "test_score", "R_MSE"] = newscore_df ["train_score", "test_score", "R_MSE"]
score_df=newscore_df
display(score_df)